In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

import gc
import os
import time
import warnings
from itertools import combinations
from warnings import simplefilter

import joblib
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, TimeSeriesSplit

warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Preparation

In [3]:
all = pd.read_csv("/content/drive/MyDrive/Desktop/Advanced Analytics Edge/Project/Individual Work/Input/train.csv")
all = all.dropna(subset=['target','wap'])
all.isnull().sum()

stock_id                         0
date_id                          0
seconds_in_bucket                0
imbalance_size                   0
imbalance_buy_sell_flag          0
reference_price                  0
matched_size                     0
far_price                  2894122
near_price                 2856960
bid_price                        0
bid_size                         0
ask_price                        0
ask_size                         0
wap                              0
target                           0
time_id                          0
row_id                           0
dtype: int64

In [4]:
all_sample = all[all["date_id"]>450]
train = all_sample[all_sample["date_id"]<475]
test = all_sample[all_sample["date_id"]>=475]
del all, all_sample

print(train.shape)
print(test.shape)
train.head()

(264000, 17)
(66000, 17)


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
4907980,0,451,0,1.016637e+08,-1,1.001570,27901114.55,NaN,NaN,0.999892,56601.00,1.000060,31225.16,1.0,12.749434,24805,451_0_0
4907981,1,451,0,1.481195e+06,1,0.999733,2922825.96,NaN,NaN,0.999782,20602.00,1.000218,20611.00,1.0,-0.090003,24805,451_0_1
4907982,2,451,0,0.000000e+00,0,0.999531,7211807.46,NaN,NaN,0.999480,19685.00,1.000546,20691.30,1.0,5.480051,24805,451_0_2
4907983,3,451,0,5.426093e+06,-1,1.000054,52742546.23,NaN,NaN,0.999902,10059.75,1.000104,10653.66,1.0,-3.870130,24805,451_0_3
4907984,4,451,0,6.102518e+06,-1,1.000013,10336604.95,NaN,NaN,0.998472,5346.00,1.000153,535.50,1.0,2.980232,24805,451_0_4


In [5]:
weights = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
]

weights = {int(k):v for k,v in enumerate(weights)}

# Feature Engineering

In [6]:
global_stock_id_feats = {
        "median_size": train.groupby("stock_id")["bid_size"].median() + train.groupby("stock_id")["ask_size"].median(),
        "std_size": train.groupby("stock_id")["bid_size"].std() + train.groupby("stock_id")["ask_size"].std(),
        "ptp_size": train.groupby("stock_id")["bid_size"].max() - train.groupby("stock_id")["bid_size"].min(),
        "median_price": train.groupby("stock_id")["bid_price"].median() + train.groupby("stock_id")["ask_price"].median(),
        "std_price": train.groupby("stock_id")["bid_price"].std() + train.groupby("stock_id")["ask_price"].std(),
        "ptp_price": train.groupby("stock_id")["bid_price"].max() - train.groupby("stock_id")["ask_price"].min(),
    }

In [7]:
from numba import njit, prange

@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            if mid_val == min_val:  # Prevent division by zero
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features


def calculate_triplet_imbalance_numba(price, df):
    # Convert DataFrame to numpy array for Numba compatibility
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Calculate the triplet imbalance
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Create a DataFrame from the results
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features

In [8]:
# generate imbalance features
def imbalance_features(df):
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]

    # V1
    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    df["size_imbalance"] = df.eval("bid_size / ask_size")

    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")

    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values

    # V2
    df["stock_weights"] = df["stock_id"].map(weights)
    df["weighted_wap"] = df["stock_weights"] * df["wap"]
    df['wap_momentum'] = df.groupby('stock_id')['weighted_wap'].pct_change(periods=6)
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    df['spread_depth_ratio'] = (df['ask_price'] - df['bid_price']) / (df['bid_size'] + df['ask_size'])
    df['mid_price_movement'] = df['mid_price'].diff(periods=5).apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    df['micro_price'] = ((df['bid_price'] * df['ask_size']) + (df['ask_price'] * df['bid_size'])) / (df['bid_size'] + df['ask_size'])
    df['relative_spread'] = (df['ask_price'] - df['bid_price']) / df['wap']

    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)

    # V3
    for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
        for window in [1, 2, 3, 5, 10]:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window)

    for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size',
                'wap', 'near_price', 'far_price']:
        for window in [1, 2, 3, 5, 10]:
            df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window)

    return df.replace([np.inf, -np.inf], 0)

# generate time & stock features
def other_features(df):
    df["dow"] = df["date_id"] % 5
    df["dom"] = df["date_id"] % 20
    df["seconds"] = df["seconds_in_bucket"] % 60
    df["minute"] = df["seconds_in_bucket"] // 60

    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict())

    return df

# generate all features
def generate_all_features(df):
    cols = [c for c in df.columns if c not in ["row_id", "time_id"]]
    df = df[cols]
    df = imbalance_features(df)
    df = other_features(df)
    gc.collect()

    feature_name = [i for i in df.columns if i not in ["row_id", "time_id", "date_id"]]

    return df[feature_name]

In [9]:
# Generate data set
train = generate_all_features(train)
print("Build Train Finished.")
test = generate_all_features(test)
print("Build Test Finished.")

# Preparing the data
X_train = train.drop(columns=['target'])
X_test = test.drop(columns=['target'])
y_train = train['target']
y_test = test['target']

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# get non-missing features
feature_no_missing = train.isnull().sum() == 0
X_train_scaled = X_train_scaled.loc[:, feature_no_missing]
X_test_scaled = X_test_scaled.loc[:, feature_no_missing]

Build Train Finished.
Build Test Finished.


In [10]:
# Get Prediction result
def predict_result(X_train, X_test, y_train, y_test, model):

    # Make predictions on the training set
    train_predictions = model.predict(X_train)
    # Flatten predictions if necessary
    train_predictions = np.ravel(train_predictions)

    # Make predictions on the test set
    test_predictions = model.predict(X_test)
    # Flatten predictions if necessary
    test_predictions = np.ravel(test_predictions)

    # Calculate Mean Absolute Error on training data
    train_mae = mean_absolute_error(y_train, train_predictions)
    # Calculate Mean Absolute Error
    test_mae = mean_absolute_error(y_test, test_predictions)

    return train_predictions, test_predictions, train_mae, test_mae

In [11]:
# Calculate test MAE
def calculate_mae(y_test, y_test_pred):

    # Calculate Mean Absolute Error
    test_mae = mean_absolute_error(y_test, y_test_pred)

    return test_mae

# Calculate MAE

## Neural Network

In [44]:
# read prediction results
nn_train_pred = pd.read_csv('/content/drive/MyDrive/Desktop/Advanced Analytics Edge/Project/Individual Work/Prediction/nn_train_pred.csv', header=None)
dnn_train_pred = pd.read_csv('/content/drive/MyDrive/Desktop/Advanced Analytics Edge/Project/Individual Work/Prediction/dnn_train_pred.csv', header=None)
nn_test_pred = pd.read_csv('/content/drive/MyDrive/Desktop/Advanced Analytics Edge/Project/Individual Work/Prediction/nn_test_pred.csv', header=None)
dnn_test_pred = pd.read_csv('/content/drive/MyDrive/Desktop/Advanced Analytics Edge/Project/Individual Work/Prediction/dnn_test_pred.csv', header=None)

In [45]:
nn_test_mae = calculate_mae(y_test, nn_test_pred)
dnn_test_mae = calculate_mae(y_test, dnn_test_pred)
print("Neural Network MAE: ", nn_test_mae)
print("Deep Neural Network MAE: ", dnn_test_mae)

Neural Network MAE:  5.314431949579221
Deep Neural Network MAE:  5.324659005868376


## ARIMA

In [14]:
# read prediction results
arima_train_pred = pd.read_csv('/content/drive/MyDrive/Desktop/Advanced Analytics Edge/Project/Individual Work/Prediction/arima_train_pred.csv', header=None)
arima_test_pred = pd.read_csv('/content/drive/MyDrive/Desktop/Advanced Analytics Edge/Project/Individual Work/Prediction/arima_test_pred.csv', header=None)

In [15]:
arima_test_mae = calculate_mae(y_test, arima_test_pred)
print("ARIMA MAE: ", arima_test_mae)

ARIMA MAE:  5.451579808512577


## Random Forest

In [16]:
# read prediction results
rf_train_pred = pd.read_csv('/content/drive/MyDrive/Desktop/Advanced Analytics Edge/Project/Individual Work/Prediction/rf_train_pred.csv', header=None)
rf_test_pred = pd.read_csv('/content/drive/MyDrive/Desktop/Advanced Analytics Edge/Project/Individual Work/Prediction/rf_test_pred.csv', header=None)

In [17]:
rf_test_mae = calculate_mae(y_test, rf_test_pred)
print("ARIMA MAE: ", rf_test_mae)

ARIMA MAE:  5.33953604421359


## XGBoost

In [18]:
# read prediction results
xgb_train_pred = pd.read_csv('/content/drive/MyDrive/Desktop/Advanced Analytics Edge/Project/Individual Work/Prediction/xgb_train_pred.csv', header=None)
xgb_test_pred = pd.read_csv('/content/drive/MyDrive/Desktop/Advanced Analytics Edge/Project/Individual Work/Prediction/xgb_test_pred.csv', header=None)

In [19]:
xgb_test_mae = calculate_mae(y_test, xgb_test_pred)
print("XGBoost MAE: ", xgb_test_mae)

XGBoost MAE:  5.413000145072318


## LightGBM

In [20]:
# read prediction results
lgbm_train_pred = pd.read_csv('/content/drive/MyDrive/Desktop/Advanced Analytics Edge/Project/Individual Work/Prediction/lgbm_train_pred.csv', header=None)
lgbm_test_pred = pd.read_csv('/content/drive/MyDrive/Desktop/Advanced Analytics Edge/Project/Individual Work/Prediction/lgbm_test_pred.csv', header=None)

In [21]:
lgbm_test_mae = calculate_mae(y_test, lgbm_test_pred)
print("LightGBM MAE: ", lgbm_test_mae)

LightGBM MAE:  5.314149372953361


## Combine Results

In [22]:
# # combine train results
# train_pred = pd.concat([nn_train_pred, dnn_train_pred, arima_train_pred, rf_train_pred, xgb_train_pred, lgbm_train_pred], axis=1)
# train_pred.columns = ['nn_pred', 'dnn_pred', 'arima_pred', 'rf_pred', 'xgb_pred', 'lgbm_pred']
# train_pred.head()

# # combine test results
# test_pred = pd.concat([nn_test_pred, dnn_test_pred, arima_test_pred, rf_test_pred, xgb_test_pred, lgbm_test_pred], axis=1)
# test_pred.columns = ['nn_pred', 'dnn_pred', 'arima_pred', 'rf_pred', 'xgb_pred', 'lgbm_pred']
# test_pred.head()

In [51]:
# combine train results
train_pred = pd.concat([arima_train_pred, xgb_train_pred], axis=1)
train_pred.columns = ['arima_pred', 'xgb_pred']
train_pred.head()

# combine test results
test_pred = pd.concat([arima_test_pred, xgb_test_pred], axis=1)
test_pred.columns = ['arima_pred', 'xgb_pred']
test_pred.head()

,arima_pred,xgb_pred
0,0.651118,1.902638
1,2.830689,-11.661285
2,-2.414776,1.523604
3,-2.323376,0.831036
4,-2.423109,-2.153650


# Assign weights by regression model

In [52]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [53]:
train_pred

,arima_pred,xgb_pred
0,0.000000,8.445658
1,0.000000,-3.490235
2,0.000000,1.354029
3,0.000000,0.484419
4,0.000000,-1.786226
...,...,...
263995,6.482602,-1.537830
263996,-5.845439,0.621637
263997,-12.002358,-2.759497
263998,-3.853818,0.553993


In [54]:
lg_model = LinearRegression()
lg_model.fit(train_pred, y_train)
lg_model.coef_

array([0.61151547, 0.84933884])

In [55]:
ensemble_pred = lg_model.predict(test_pred)
ensemble_test_mae = calculate_mae(y_test, ensemble_pred)
print("ensemble MAE: ", ensemble_test_mae)

ensemble MAE:  5.411070778747745


In [56]:
mean_train = np.mean(y_train)

In [29]:
ss_tot = np.sum((y_train - mean_train)**2)

In [35]:
y_train - np.array(arima_train_pred).flatten()

4907980    12.749434
4907981    -0.090003
4907982     5.480051
4907983    -3.870130
4907984     2.980232
             ...    
5171975    -8.802414
5171976     2.815735
5171977    -0.977746
5171978     2.403637
5171979    -3.669858
Name: target, Length: 264000, dtype: float64

In [39]:
# calculate R^2
mean_train = np.mean(y_train)
ss_tot = np.sum((y_train - mean_train)**2)
ss_res = np.sum((y_train - np.array(lgbm_train_pred).flatten())**2)
r2 = 1 - ss_res/ss_tot
print("R^2: ", r2)

R^2:  0.23835319711946412


In [43]:
# calculate of-sample R^2
mean_test = np.mean(y_test)
ss_tot = np.sum((y_test - mean_train)**2)
ss_res = np.sum((y_test - np.array(rf_test_pred).flatten())**2)
r2 = 1 - ss_res/ss_tot
print("OSR^2: ", )

OSR^2:  0.0061594231921707765
